未完成整理：
* 数据采集／获取
* Tensorflow内置的Machine Learning部分函数
* 学习资源汇总

## 第一章的学习要点

 * 用来将数据正则化：data = tf.nn.batch_norm_with_global_normalization(...)

【问题】：正则化就是我所理解的那个正则化吗？（减去期待值后再除以方差）
 
 * 标准建模的流程：
 
 import/generate datasets ---> Transform and normalize data ---> Partition datasets into train,test and validation sets ---> set algorithm parameters(hyperparameters) ---> initialize variables and placeholders --->define model structure ---> declare the loss function ---> initialize and train the model ---> evaluate the model ---> tune hyperparameters ---> deploy/predict new outcomes
 

In [1]:
import tensorflow as tf
import numpy as np

### 生成一般Tensor

In [4]:
#生成一个全是0的Tensor
zero_tensor = tf.zeros([3,3])

In [3]:
#生成一个全是1的Tensor
ones_tensor = tf.ones([3,3])

In [12]:
#生成一个全是特定常数的Tensor
filled_tensor = tf.fill([3,3],30)

Tensor("Fill_1:0", shape=(3, 3), dtype=int32)


In [8]:
#根据一个已经存在的常量生成Tensor
constant_tensor = tf.constant([[1,2,3],[4,5,6]])

In [14]:
#根据一个已有的Tensor的结构来生成一个Tensor
zeros_similar = tf.zeros_like(constant_tensor)
ones_similar = tf.ones_like(constant_tensor)
ones_compare = tf.fill([2,3],1)
print(ones_similar)
print(ones_compare)
#从下面那个比较可知比较Tensor时应该也是类似于比较对象，仅仅是所有属性相同并不等于相等
#应该时对象间的比较
print(ones_similar == ones_compare)

Tensor("ones_like_4:0", shape=(2, 3), dtype=int32)
Tensor("Fill_3:0", shape=(2, 3), dtype=int32)
False


### 生成序列Tensor

In [19]:
linear_tensor = tf.linspace(0.0,1.0,5)
#上述函数的包含了终止数值，即右包含
linear_seq_tensor = tf.range(6,15,3)
#range的话就不包含右边的数值

### 生成随机Tensor

In [21]:
#生成uniform distribution
#生成的随机数值不包含maxval
randunif_tensor = tf.random_uniform([3,3],minval=0,maxval=1)

In [24]:
#生成normal distribution
randnorm_tensor = tf.random_normal([3,3],mean=0,stddev=1)

In [29]:
#生成受限的normal distribution
#生成的是两个标准差之内的数值
runcnorm_tensor = tf.truncated_normal([3,3],mean=0, stddev=1)

【问题】：suhffle到底shuffle的是什么东西来着……有点忘了
【问题】：crop也是谜一般的玩意

In [32]:
#random shuffle
shuffled_output = tf.random_shuffle(runcnorm_tensor)
#crop Tensor

#random_crop

### 生成变量

In [33]:
my_var = tf.Variable(tf.zeros([3,3]))

### 将各种形式的变量转换为Tensor的万能函数（？）
【问题】：为什么能在convert_to_tensor的参数里面使用Tensor？

In [35]:
new_var = tf.convert_to_tensor([1,2,3])
print(new_var)

Tensor("Const_7:0", shape=(3,), dtype=int32)


### 变量的初始化

In [36]:
#所有的Variable在使用前都必须进行初始化，每个Variable自身都有initializer，但是每个
#Variable都单独进行初始化有时候比较麻烦所以用global_variable_initializer也比较
#常见
my_var = tf.Variable(tf.zeros(2,3))
sess = tf.Session()
initialize_op = tf.global_variables_initializer()
sess.run(initialize_op)

In [37]:
#但是有的时候如果其中一个Tensor的结构依赖于另外一个Tensor，就必须进行单独的初始化
sess = tf.Session()
first_val = tf.Variable(tf.zeros([2,3]))
sess.run(first_val.initializer)
second_val = tf.Variable(tf.zeros_like(first_val))
sess.run(second_val.initializer)

### Placeholder使用方法

In [9]:
sess = tf.Session()
x = tf.placeholder(tf.float32,shape=[2,2])
y = tf.identity(x)
x_vals = np.random.rand(2,2)
print(sess.run(x,feed_dict={x:x_vals}))
#如果上面写成sess.run(x,feed_dict={x:x_vals})也不会出循环引用的问题？可能Tensorflow
#后面几个版本有进行优化吧

[[ 0.4646773   0.75074887]
 [ 0.04458335  0.01987305]]


### 生成matrices&matrix各种特性

In [10]:
identity_matrix = tf.diag([1.0,1.0,1.0])

In [11]:
A = tf.truncated_normal([2,3])
B = tf.random_uniform([3,2])
C = tf.fill([3,3],5.0)
D = tf.convert_to_tensor([[1.,2.,3.],[-3.,-7.,-1.],[0.,5.,-2.]])

In [12]:
print(sess.run(identity_matrix))

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]


In [14]:
#矩阵乘法
print(sess.run(tf.matmul(identity_matrix,C)))

[[ 5.  5.  5.]
 [ 5.  5.  5.]
 [ 5.  5.  5.]]


In [17]:
#矩阵转置
#每次sess.run都会重新生成一次Tensor
print(sess.run(A))
print(sess.run(tf.matmul(tf.transpose(A),A)))

[[-1.21678245  0.19870323  1.13959551]
 [-0.04773329 -0.52218384 -0.05092119]]
[[ 0.01096273  0.00436912 -0.01823799]
 [ 0.00436912  1.18608701 -1.56131327]
 [-0.01823799 -1.56131327  2.06948829]]


In [25]:
#determinant and inverse
#Tensorflow的inverse在matrix正定的时候是基于Cholesky分解，其他时候是基于LU分界的
print(sess.run(tf.matrix_inverse(D)))
print("----------------")
print(sess.run(tf.matmul(tf.matrix_inverse(D),D)))
print(sess.run(tf.matrix_determinant(D)))

[[-0.50000006 -0.5        -0.50000006]
 [ 0.15789475  0.05263158  0.21052633]
 [ 0.39473686  0.13157895  0.0263158 ]]
----------------
[[  9.99999940e-01  -2.38418579e-07  -1.19209290e-07]
 [  0.00000000e+00   1.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   2.98023224e-08   1.00000000e+00]]
-38.0


【问题】：把Cholesky分解和LU分解再看一遍

In [26]:
#分解
print(sess.run(tf.cholesky(identity_matrix)))

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]


In [31]:
#eigen特征值和特征向量
print(sess.run(tf.self_adjoint_eig(D)))
#这样得到的结果是一个tuple，其中第一项是特征值，第二项是特征向量的list，可以通过tuple分
#解取出，这种tuple的结构叫做Eigen decomposition of a matrix
eigen_value, eigen_vector = tf.self_adjoint_eig(D)
print(sess.run(eigen_value))
print(sess.run(eigen_vector))

(array([-10.65907574,  -0.22750695,   2.88658309], dtype=float32), array([[-0.21749546, -0.63250113, -0.74339646],
       [-0.84526515, -0.25879988,  0.46749276],
       [ 0.48808062, -0.7300446 ,  0.47834331]], dtype=float32))
[-10.65907574  -0.22750695   2.88658309]
[[-0.21749546 -0.63250113 -0.74339646]
 [-0.84526515 -0.25879988  0.46749276]
 [ 0.48808062 -0.7300446   0.47834331]]


### Tensor基本运算符（其实是函数）

In [39]:
#div默认返回参数类型，所以参数类型为Int的时候返回值也会为Int，所以要养成加.的好习惯
#类似于Python3的除法要用truediv
print(sess.run(tf.div(3,4)))
print(sess.run(tf.div(3.,4)))
print("-------------------")
print(sess.run(tf.truediv(3,4)))

0
0.75
-------------------
0.75


In [40]:
#floor
#返回值依然依据参数类型，但是会向下取整到最大的整数
print(sess.run(tf.floordiv(3.,4)))

0.0


In [42]:
#mod
#返回值依据参数类型
print(sess.run(tf.mod(22.,5.)))
print(sess.run(tf.mod(22,5)))

2.0
2


其余常用的函数：
 * abs()
 * ceil()
 * cos()
 * exp()
 * floor()
 * inv()
 * log()
 * maximum()
 * minimum()
 * neg()
 * pow()
 * round()
 * rsqrt()
 * sign()
 * sin()
 * sqrt()
 * square()
 * cross()
 
 注意所有的函数都是element-wise的，以maximum为例的话，就是返回的Tensor中的每一个
 dimension上的值都是该dimension上的最大值
 
 【问题】：cross到底是什么玩意……

常用于Machine Learning的一些内置函数：
 * digamma
 * erf
 * erfc
 * igamma
 * igammac
 * lbeta
 * lgamma
 * squared_difference
 
 这个感觉好复杂了……之后又时间再整理吧
 
 【问题】：把上述函数的作用和原理刷一遍……

#### 虽然Tensorflow里内置了很多函数，但是更多情况下都是要基于这些基本函数来生成自己真正需要的函数

In [44]:
#生成一个polynomial函数
def custom_polynomial(value):
    return tf.subtract(3 * tf.square(value), value) + 10
print(sess.run(custom_polynomial(5)))
#一个重要的细节：老版本的Tensorflow中的减法是sub，但是新版本换成了substract

80


### Activation Function

激活函数是内置于nn库的

In [45]:
#最常见的激活函数ReLU
print(sess.run(tf.nn.relu([-3,3,10])))

[ 0  3 10]


In [46]:
#也可以给ReLU函数的正方向加个Cap，函数表示的话就是min(max(0,x),Cap),这在nn库中的
#实现就是relu6
print(sess.run(tf.nn.relu6([-3,3,10])))
#relu6是一种hard-sigmoid函数，好处是计算更快而且不容易出现vanishing或exploding

[0 3 6]


【问题】：仔细思考为什么relu6有上述的那些好处，搞清楚hard-sigmoid到底是什么定义

In [48]:
#最常用／老出问题的激活函数sigmoid function/logistic function
#函数表示：1/(1+exp(-x))
#返回值永远在0和1之间
#老是出现vanishing
print(sess.run(tf.nn.sigmoid([-1.,0.,1.])))
#重要tips：sigmoid使用的时候参数只能是float或者double，传入int的话会出错！

[ 0.26894143  0.5         0.7310586 ]


In [50]:
#不那么长用／姑且记一下的激活函数hyperbolic tangent
#函数表示：(exp(x) - exp(-x))/(exp(x) + exp(-x))
print(sess.run(tf.nn.tanh([-1.,0,1])))
#与sigmoid函数相同，参数只能是float或者double，但是list的时候只要其中任何一个元素的
#类型是float／double，整个list都会强制升级

[-0.76159418  0.          0.76159418]


In [52]:
#第一次见的激活函数softsign
#如名字所提示，是sign的连续近似型
#函数表示：x/abs(x)+1
print(sess.run(tf.nn.softsign([-1.,0,1])))
#允许传入int，但是传入后返回值也变成int了……算了以后还是老老实实所有list都变符点吧……

[-0.5  0.   0.5]


In [53]:
#同～第一次见的激活函数softplus
#函数表示：log(exp(x)+1)
print(sess.run(tf.nn.softplus([-1.,0,1])))

[ 0.31326166  0.69314718  1.31326163]


In [54]:
#意外的经常见到的激活函数Exponential Linear Unit(ELU)
#感觉算是ReLU的改良型之一？
#函数表示：(exp(x)+1) if x<0 else x
print(sess.run(tf.nn.elu([-1.,0,1])))

[-0.63212055  0.          1.        ]


## 重要！！activation function的选择

activation function的选择要依据所传入参数的类型，一般情况下我们倾向于保留参数／数据的统计学特征，所以当传入的数据期待值为0时，为了保持住variance就很明显不能使用sigmoid（sigmoid的返回值是0到1，会大量改变数据特征），更好的选择是tanh或者softsign

ざっくり分類してみよう！

* ReLU系：ReLU,ReLU6,Softplus,ELU
* Sigmoid系：Sigmoid,Tanh,Softsign